In [1]:
#librerias para el tratamiento de la API, transformacion en data frame, y tratamiento de fechas
import requests
import pandas as pd
from datetime import datetime, timedelta
import numpy as np
import re

#librerias para conectar la tabla con redshift
import psycopg2

#libreria para tratar las credenciales
import json


In [2]:
# Leer el archivo de credenciales
with open('credentials.json') as file:
    credentials = json.load(file)

In [5]:
#Parametros de la busqueda

query = '''ucrania OR Ukraine OR Ukraine OR Ucraina'''#palabras clave para efectuar la busqueda en API

today = datetime.today()
yesterday = today - timedelta(days=1) # Obtener la fecha del dia de ayer
from_date = yesterday.strftime('%Y-%m-%d')
results_limit = 100 
api_key = credentials['api_key']
languages = ['es', 'en', 'fr', 'it']

# Funcion para realizar la conexion a la API
def get_articles(url):
    try:
        response = requests.get(url)
        response.raise_for_status()
    except requests.exceptions.RequestException as e:
        print('Error al conectarse a la API:', e)
        return []
    else:
        data = response.json()
        articles = data['articles']
        return articles

# Crear una lista de diccionarios con los datos de cada artículo.
articles_list = []
for lang in languages:
    url = f"https://newsapi.org/v2/everything?q={query}&language={lang}&from={from_date}&pageSize={results_limit}&apiKey={api_key}"
    articles = get_articles(url)
    for article in articles:
        article_data = {
            'author': article['author'],
            'title': article['title'],
            'description': article['description'],
            'url': article['url'],
            'publishedAt': article['publishedAt'],
            'source': article['source']['name'],
            'language': lang
                        }
        articles_list.append(article_data)


In [6]:
#convertir la lista de diccionarios en un DataFrame
df = pd.DataFrame(articles_list)

In [7]:
#Variables y funciones generales
#funcion para transformar las siglas de la columna language
def transformar_siglas_a_idiomas(df, columna):
    siglas_a_idiomas = {
        'en': 'english',
        'fr': 'french',
        'it': 'italian',
        'es': 'spanish'
    }
    df[columna] = df[columna].map(siglas_a_idiomas)
    return df

#Base para reordenar las columnas
column_order = ['source', 'title', 'description','author',  'publishedAt','language', 'url']

def limpiar_celdas(columna):
    #Excepciones : , . numeros vocoales con tilde /
    patron = r"[^a-zA-ZáéíóúÁÉÍÓÚüÜñÑ.,\d\s/]" #Patrón de expresión regular para encontrar caracteres no deseados
    columna = columna.apply(lambda x: re.sub(patron, '', str(x)))  # Aplicar el reemplazo a cada celda de la columna
    return columna

def eliminar_parentesis(columna):
    patron = r'\((.*?)\)'  # Patrón de expresión regular para encontrar el texto entre paréntesis
    columna = columna.apply(lambda x: re.sub(patron, '', x)).str.strip()
    # Eliminar el texto entre paréntesis y eliminar espacios en blanco
    return columna

In [8]:
news_cleaned = df

In [9]:
#Limpieza general del DF

#algunas filas son iguales entre si, con la excepcion del url. Elimino duplicados sin tener en cuenta el link.
news_cleaned = news_cleaned.drop_duplicates(subset=['source', 'title', 'description', 'author', 'publishedAt', 'language'])
# Paso a lowercase todas las palabras, a fin de evitar inconsistencias o discrepancias a la hora 
# de hacer un analisis de los textos
news_cleaned = news_cleaned.apply(lambda x: x.astype(str).str.lower())
#ejecuto funcion para tranformar las siglas en idiomas en la columa 'languages'
news_cleaned = transformar_siglas_a_idiomas(news_cleaned, 'language')
#reemplazo los valores vacios por Nan
news_cleaned = news_cleaned.replace('', np.nan)
#Reemplazo los Nan values (en el caso de la columna 'author', busco el diario que publico la nota)
#En el caso de la descripcion sobre la nota, dejo el apartado 'no data'
news_cleaned['author'] = news_cleaned['author'].fillna(news_cleaned['source'])
news_cleaned['description'] = news_cleaned['description'].fillna('no data')
#Reordeno las columnas
news_cleaned = news_cleaned.reindex(columns=column_order)

In [10]:
#Limpieza especifica por columna

#Columna source
news_cleaned['source'] = eliminar_parentesis(news_cleaned['source'])

#Columna authors
#algunas celdas figuran con 'none'. Misma solucion que con los Nan Values, reemplazo el none
#por el medio periodistico
mask = news_cleaned['author'].eq('none')
news_cleaned['author'] = news_cleaned['author'].where(~mask, news_cleaned['source'])
#Muchos autores figuran con la siguiente descripcion: la nacion (Carlos pagni). Lo que hago aca
#es dejar unicamente los nombres que figuran entre parentesis
mask = news_cleaned['author'].str.contains(r'\([^)]+\)')
news_cleaned.loc[mask, 'author'] = news_cleaned.loc[mask, 'author'].str.extract(r'\(([^)]+)\)', expand=False)
#Ejecuto funcion para limpiar columna de caracteres no alfabeticos (salvo excepciones)
news_cleaned['author'] = limpiar_celdas(news_cleaned['author'])
#ajusto casos puntuales mediante replace
news_cleaned['author'] = (
    news_cleaned['author']
    .str.replace('rt en español\n', 'rt en español')
    .str.replace('https//www.facebook.com/bbcnews', 'bbcnews')
    .str.replace('rt en español , rt en español', 'rt en español')
)

#columna publishedAt
#elimino caracteres sobrantes, quedando unicamente YYYY-MM-DD
news_cleaned['publishedAt'] = news_cleaned['publishedAt'].str[:10]

In [12]:
# Realizar conexion a Redshift mediante psycopg2
try:
    conn = psycopg2.connect(
    host='data-engineer-cluster.cyhh5bfevlmn.us-east-1.redshift.amazonaws.com',
    dbname=credentials['db_username'],
    user=credentials['db_password'],
    password='qjA21aB81Y',
    port='5439'
)
    print("Conexion con Redshift exitosa")
    
except Exception as e:
    print("Error al conectar con Redshift")
    print(e)


Conexion con Redshift exitosa


In [14]:
# Crear un cursor para ejecutar sentencias SQL
cur = conn.cursor()

In [15]:
# Eliminar la tabla si ya existe. Ejecutar solo en caso de error
#cur.execute('DROP TABLE IF EXISTS news_articles')

# Crear una tabla en Redshift, en caso de que no exista
table_name = 'news_articles'
cur.execute(f'''
CREATE TABLE IF NOT EXISTS {table_name} (
    id INT IDENTITY(1, 1),
    source VARCHAR(255),
    title VARCHAR(1000),
    description VARCHAR(1000),
    author VARCHAR(255),
    publishedAt DATE,
    language VARCHAR(7),
    url NVARCHAR(1000)
)
''')

In [17]:
# Insertar los datos obtenidos de la API en la tabla
for index, row in news_cleaned.iterrows():
    cur.execute(f'''
        INSERT INTO {table_name} (source, title, description, author, publishedAt, language, url)
        VALUES (%s, %s, %s, %s, %s, %s, %s);
    ''', (
        row['source'],
        row['title'],
        row['description'],
        row['author'],
        row['publishedAt'],
        row['language'],
        row['url']
    ))

# Hacer commit de la transacción y cerrar la conexión
conn.commit()
conn.close()



In [ ]:
#En caso de que el codigo para insertar los datos en la tabla de redshift fallen, ejecuto:
#cur.execute('ROLLBACK')
